### Notebook to demonstrate Multitask Classification workflow

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

![image](https://developer.nvidia.com/sites/default/files/akamai/TAO/tlt-tao-toolkit-bring-your-own-model-diagram.png)


### The workflow in a nutshell

- Creating a dataset
- Upload kitti dataset to the service
- Running dataset convert
- Getting a PTM from NGC
- Model Actions
    - Train
    - Evaluate
    - Prune, retrain
    - Export
    - Convert
    - Inference on TAO

### Table of contents

1. [Create datasets ](#head-1)
1. [List the created datasets](#head-2)
1. [Create model ](#head-4)
1. [List models](#head-5)
1. [Assign train, eval datasets](#head-6)
1. [Assign PTM](#head-7)
1. [Actions](#head-8)
1. [Train](#head-9)
1. [Evaluate](#head-10)
1. [Optimize: Apply specs for prune](#head-12)
1. [Optimize: Apply specs for retrain](#head-13)
1. [Optimize: Run actions](#head-14)
1. [Export: FP32](#head-15)
1. [Export: INT8](#head-16)
1. [Model convert using TAO-Converter](#head-17)
1. [TAO inference](#head-18)

### Requirements
Please find the server requirements [here](https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_api/api_setup.html#)

In [ ]:
import json
import os
import requests
import uuid
import time
from IPython.display import clear_output

### FIXME

1. Assign a workdir in FIXME1
2. Assign the ip_address and port_number in FIXME 2 ([info](https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_api/api_rest_api.html))
3. Assign the ngc_api_key variable in FIXME 3
4. Choose between default and custom dataset in FIXME 4

In [ ]:
# Define workspaces and other variables
workdir = "workdir_multitask_classification" #FIXME1
host_url = "http://<ip_address>:<port_number>" # FIXME2 example: https://10.137.149.22:32334
# In host machine, node ip_address and port number can be obtained as follows,
# ip_address: hostname -i
# port_number: kubectl get service ingress-nginx-controller -o jsonpath='{.spec.ports[0].nodePort}'
ngc_api_key = "<ngc_api_key>" # FIXME3 example: zZYtczM5amdtdDcwNjk0cnA2bGU2bXQ3bnQ6NmQ4NjNhMDItMTdmZS00Y2QxLWI2ZjktNmE5M2YxZTc0OGyM

In [ ]:
# Exchange NGC_API_KEY for JWT
response = requests.get(f"{host_url}/api/v1/login/{ngc_api_key}")
user_id = response.json()["user_id"]
print("User ID",user_id)
token = response.json()["token"]
print("JWT",token)

# Set base URL
base_url = f"{host_url}/api/v1/user/{user_id}"
print("API Calls will be forwarded to",base_url)

headers = {"Authorization": f"Bearer {token}"}

In [ ]:
# Creating workdir
if not os.path.isdir(workdir):
    os.makedirs(workdir)

### Create datasets <a class="anchor" id="head-1"></a>

We will be using the `Fashion Product Images (Small)` for the tutorial. This dataset is available on Kaggle.In this tutorial, our trained classification network will perform three tasks: article category classification, base color classification and target season classification. 

 To download the dataset, you will need a Kaggle account. After login, you can download the dataset zip file [here](https://www.kaggle.com/paramaggarwal/fashion-product-images-small). The downloaded file is archive.zip with a subfolder called myntradataset. Unzip contents in this subfolder to your workdir created in the cell above and you should have a folder called images and a CSV file called styles.csv

**If using custom dataset; it should follow this dataset structure**
```
DATA_DIR
├── images
│   ├── image_name_1.jpg
│   ├── image_name_2.jpg
|   |   ├── ...
├── styles.csv
```

In [ ]:
DATA_DIR = "mt_class_data"
os.environ['DATA_DIR']= DATA_DIR
!mkdir -p $DATA_DIR

In [ ]:
dataset_to_be_used = "default" #FIXME4 #default/custom; default for the dataset used in this tutorial notebook; custom for a different dataset

In [ ]:
if dataset_to_be_used == "default":
    !unzip -uq $DATA_DIR/archive.zip -d $DATA_DIR/
    # Check the dataset is present
    !if [ ! -d $DATA_DIR/images ]; then echo 'images folder NOT found.'; else echo 'Found images folder.';fi
    !if [ ! -f $DATA_DIR/styles.csv ]; then echo 'CSV file NOT found.'; else echo 'Found CSV file.';fi

In [ ]:
# Create subdirectories and remove existing files in them
!mkdir -p $DATA_DIR/images_train && rm -rf $DATA_DIR/images_train/*
!mkdir -p $DATA_DIR/images_val && rm -rf $DATA_DIR/images_val/*
!mkdir -p $DATA_DIR/images_test && rm -rf $DATA_DIR/images_test/*

In [ ]:
!python3 -m pip install numpy
!python3 -m pip install pandas
import os
import shutil
import numpy as np
import pandas as pd

df = pd.read_csv(os.environ['DATA_DIR'] + '/styles.csv', error_bad_lines=False, warn_bad_lines=False)
df = df[['id', 'baseColour', 'subCategory', 'season']]
df = df.dropna()
category_cls = df.subCategory.value_counts()[:10].index # 10-class multitask-classification
season_cls = ['Spring', 'Summer', 'Fall', 'Winter'] # 4-class multitask-classification
color_cls = df.baseColour.value_counts()[:11].index # 11-class multitask-classification

# Get all valid rows
df = df[df.subCategory.isin(category_cls) & df.season.isin(season_cls) & df.baseColour.isin(color_cls)]
df.columns = ['fname', 'base_color', 'category', 'season']
df.fname = df.fname.astype(str)
df.fname = df.fname + '.jpg'

# remove entries whose image file is missing
all_img_files = os.listdir(os.environ['DATA_DIR'] + '/images')
df = df[df.fname.isin(all_img_files)]

idx = np.arange(len(df))
np.random.shuffle(idx)

train_split_idx = int(len(df)*0.8)
train_df = df.iloc[idx[:train_split_idx]]
val_df = df.iloc[idx[train_split_idx:train_split_idx+(len(df) // 10)]]
test_df = df.iloc[idx[train_split_idx+(len(df) // 10):]]

# Add a simple sanity check
assert len(train_df.season.unique()) == 4 and len(train_df.base_color.unique()) == 11 and \
    len(train_df.category.unique()) == 10, 'Training set misses some classes, re-run this cell!'
assert len(val_df.season.unique()) == 4 and len(val_df.base_color.unique()) == 11 and \
    len(val_df.category.unique()) == 10, 'Validation set misses some classes, re-run this cell!'
assert len(test_df.season.unique()) == 4 and len(test_df.base_color.unique()) == 11 and \
    len(test_df.category.unique()) == 10, 'Test set misses some classes, re-run this cell!'

for image_name in train_df["fname"]:
    source_file_name = os.path.join(DATA_DIR, "images", image_name)
    destination_file_name = os.path.join(DATA_DIR, "images_train", image_name)
    shutil.copy(source_file_name, destination_file_name)

for image_name in val_df["fname"]:
    source_file_name = os.path.join(DATA_DIR, "images", image_name)
    destination_file_name = os.path.join(DATA_DIR, "images_train", image_name)
    shutil.copy(source_file_name, destination_file_name)
    destination_file_name = os.path.join(DATA_DIR, "images_val", image_name)
    shutil.copy(source_file_name, destination_file_name)

for image_name in test_df["fname"]:
    source_file_name = os.path.join(DATA_DIR, "images", image_name)
    destination_file_name = os.path.join(DATA_DIR, "images_test", image_name)
    shutil.copy(source_file_name, destination_file_name)
    
# save processed data labels
train_df.to_csv(os.environ['DATA_DIR'] + '/train.csv', index=False)
val_df.to_csv(os.environ['DATA_DIR'] + '/val.csv', index=False)

In [ ]:
 # verify
import pandas as pd

print("Number of images in the train set. {}".format(
    len(pd.read_csv(os.environ['DATA_DIR'] + '/train.csv'))
))
print("Number of images in the validation set. {}".format(
    len(pd.read_csv(os.environ['DATA_DIR'] + '/val.csv'))
))

In [ ]:
# Sample label.
pd.read_csv(os.environ['DATA_DIR'] + '/val.csv').head()

In [ ]:
!tar -C $DATA_DIR/ -czf mt_classification_train.tar.gz images_train train.csv val.csv
!tar -C $DATA_DIR/ -czf mt_classification_val.tar.gz images_val val.csv
!tar -C $DATA_DIR/ -czf mt_classification_test.tar.gz images_test

In [ ]:
train_dataset_path =  "mt_classification_train.tar.gz"
eval_dataset_path = "mt_classification_val.tar.gz"
test_dataset_path =  "mt_classification_test.tar.gz"

In [ ]:
# Create train dataset
ds_type = "image_classification"
ds_format = "custom"
data = json.dumps({"type":ds_type,"format":ds_format})

endpoint = f"{base_url}/dataset"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(response.json())
dataset_id = response.json()["id"]

In [ ]:
# Update
dataset_information = {"name":"Train dataset",
                       "description":"My train dataset with Fashion product"}
data = json.dumps(dataset_information)

endpoint = f"{base_url}/dataset/{dataset_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

In [ ]:
# Upload
files = [("file",open(train_dataset_path,"rb"))]

endpoint = f"{base_url}/dataset/{dataset_id}/upload"

response = requests.post(endpoint, files=files, headers=headers)

print(response)
print(response.json())

In [ ]:
# Create eval dataset
ds_type = "image_classification"
ds_format = "custom"
data = json.dumps({"type":ds_type,"format":ds_format})

endpoint = f"{base_url}/dataset"

response = requests.post(endpoint,data=data, headers=headers)

print(response)
print(response.json())
eval_dataset_id = response.json()["id"]

In [ ]:
# Update
dataset_information = {"name":"Eval dataset",
                       "description":"My eval dataset with Fashion product"}
data = json.dumps(dataset_information)

endpoint = f"{base_url}/dataset/{eval_dataset_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

In [ ]:
# Upload
files = [("file",open(eval_dataset_path,"rb"))]

endpoint = f"{base_url}/dataset/{eval_dataset_id}/upload"

response = requests.post(endpoint, files=files, headers=headers)

print(response)
print(response.json())

In [ ]:
# Create testing dataset for inference
ds_type = "image_classification"
ds_format = "raw"
data = json.dumps({"type":ds_type,"format":ds_format})

endpoint = f"{base_url}/dataset"

response = requests.post(endpoint,data=data, headers=headers)

print(response)
print(response.json())
test_dataset_id = response.json()["id"]

In [ ]:
# Upload
files = [("file",open(test_dataset_path,"rb"))]

endpoint = f"{base_url}/dataset/{test_dataset_id}/upload"

response = requests.post(endpoint, files=files, headers=headers)

print(response)
print(response.json())

### List the created datasets <a class="anchor" id="head-2"></a>

In [ ]:
endpoint = f"{base_url}/dataset"

response = requests.get(endpoint, headers=headers)

print(response)
# print(response.json()) ## Uncomment for verbose list output
print("id\t\t\t\t\t type\t\t\t format\t\t name")
for rsp in response.json():
    print(rsp["id"],"\t",rsp["type"],"\t",rsp["format"],"\t\t",rsp["name"])

### Create model <a class="anchor" id="head-4"></a>

In [ ]:
network_arch = "multitask_classification"
encode_key = "tlt_encode"
data = json.dumps({"network_arch":network_arch,"encryption_key":encode_key})

endpoint = f"{base_url}/model"

response = requests.post(endpoint,data=data, headers=headers)

print(response)
print(response.json())
model_id = response.json()["id"]

### List models <a class="anchor" id="head-5"></a>

In [ ]:
endpoint = f"{base_url}/model"

response = requests.get(endpoint, headers=headers)

print(response)
# print(response.json()) ## Uncomment for verbose list output
print("model id\t\t\t     network architecture")
for rsp in response.json():
    print(rsp["id"],rsp["network_arch"])

### Assign train, eval datasets <a class="anchor" id="head-6"></a>

- Note: make sure the order for train_datasets is [source ID, target ID]
- eval_dataset is kept same as target for demo purposes
- inference_dataset is kept as target for chaining with hifigan finetune

In [ ]:
dataset_information = {"train_datasets":[dataset_id],
                       "eval_dataset":eval_dataset_id,
                       "inference_dataset":test_dataset_id}
data = json.dumps(dataset_information)

endpoint = f"{base_url}/model/{model_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

### Assign PTM <a class="anchor" id="head-7"></a>

- Search for pretrained_classification:resnet18 on NGC
- Assign it to the model

In [ ]:
# Get pretrained model for classification
model_list = f"{base_url}/model"
response = requests.get(model_list, headers=headers)

response_json = response.json()

# Search for ptm with given ngc path
ptm_id = None
for rsp in response_json:
    if rsp["ngc_path"].endswith("pretrained_classification:resnet10") and rsp["network_arch"] == network_arch:
        ptm_id = rsp["id"]
        print("Metadata for model with requested NGC Path")
        print(rsp)
        break
mt_class_ptm = ptm_id

In [ ]:
ptm_information = {"ptm":mt_class_ptm}
data = json.dumps(ptm_information)

endpoint = f"{base_url}/model/{model_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

### Actions <a class="anchor" id="head-8"></a>

For all actions:
1. Get default spec schema and derive the default values
2. Modify defaults if needed
3. Post spec dictionary to the service
4. Run model action
5. Monitor job using retrieve
6. Download results using job download endpoint (if needed)

In [ ]:
job_map = {}

### Train <a class="anchor" id="head-9"></a>

In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/train/schema"

response = requests.get(endpoint, headers=headers)

print(response)
#print(response.json()) ## Uncomment for verbose schema
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Apply changes to the specs you want to modify
specs["training_config"]["num_epochs"] = 80

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/train"

response = requests.post(endpoint,data=data, headers=headers)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

In [ ]:
# Run action
parent = None
actions = ["train"]
data = json.dumps({"job":parent,"actions":actions})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

job_map["train"] = response.json()[0]
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
job_id = job_map['train']
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    print(response)
    print(response.json())
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

In [ ]:
# Download job contents once the above job shows "Done" status
# Download output of multitask_classification train (Note: will take time)
job_id = job_map["train"]
endpoint = f'{base_url}/model/{model_id}/job/{job_id}/download'

# Save
temptar = f'{job_id}.tar.gz'
with requests.get(endpoint, headers=headers, stream=True) as r:
    r.raise_for_status()
    with open(temptar, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

print("Untarring")
# Untar to destination
tar_command = f'tar -xvf {temptar} -C {workdir}/'
os.system(tar_command)
os.remove(temptar)
print(f"Results at {workdir}/{job_id}")
model_downloaded_path = f"{workdir}/{job_id}"

In [ ]:
# Look for a model.tlt file
!ls {model_downloaded_path}/weights/

### Evaluate <a class="anchor" id="head-10"></a>

In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/evaluate/schema"

response = requests.get(endpoint, headers=headers)

print(response)
#print(response.json()) ## Uncomment for verbose schema
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Modify specs dictionary to change any config parameters

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/evaluate"

response = requests.post(endpoint,data=data, headers=headers)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

In [ ]:
# Run action
parent = job_map["train"]
actions = ["evaluate"]
data = json.dumps({"job":parent,"actions":actions})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

job_map["evaluate"] = response.json()[0]
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
job_id = job_map['evaluate']
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    print(response)
    print(response.json())
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

### Optimize <a class="anchor" id="head-11"></a>

- We optimize the trained model by pruning and retraining in the following cells

### Apply specs for prune <a class="anchor" id="head-12"></a>

In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/prune/schema"

response = requests.get(endpoint, headers=headers)

print(response)
#print(response.json()) ## Uncomment for verbose schema
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Apply changes
# None for prune

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/prune"

response = requests.post(endpoint,data=data, headers=headers)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

### Apply specs for retrain <a class="anchor" id="head-13"></a>

In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/retrain/schema"

response = requests.get(endpoint, headers=headers)

print(response)
#print(response.json()) ## Uncomment for verbose schema
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Apply changes to the specs you want to modify
specs["training_config"]["num_epochs"] = 80

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/retrain"

response = requests.post(endpoint,data=data, headers=headers)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

### Run Actions <a class="anchor" id="head-14"></a>

We use the API's job chaining feature to prune, retrain and evaluate the retrained model

In [ ]:
# Run actions
parent = job_map["train"]
actions = ["prune","retrain","evaluate"]
data = json.dumps({"job":parent,"actions":actions})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

job_map["prune"] = response.json()[0]
job_map["retrain"] = response.json()[1]
job_map["eval_retrain"] = response.json()[2]
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell (prune)
job_id = job_map['prune']
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    print(response)
    print(response.json())
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

In [ ]:
# Monitor job status by repeatedly running this cell (retrain)
job_id = job_map['retrain']
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    print(response)
    print(response.json())
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

In [ ]:
# Monitor job status by repeatedly running this cell (evaluate)
job_id = job_map['eval_retrain']
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    print(response)
    print(response.json())
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

In [ ]:
# Optional cancel job - for jobs that are pending/running (retrain)

# job_id = job_map['retrain']
# endpoint = f"{base_url}/model/{model_id}/job/{job_id}/cancel"

# response = requests.post(endpoint, headers=headers)

# print(response)
# print(response.json())

In [ ]:
# Optional delete job - for jobs that are error/done (retrain)

# job_id = job_map['retrain']
# endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

# response = requests.delete(endpoint, headers=headers)

# print(response)
# print(response.json())

### Export: FP32 <a class="anchor" id="head-15"></a>

In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/export/schema"

response = requests.get(endpoint, headers=headers)

print(response)
#print(response.json()) ## Uncomment for verbose schema
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Apply changes
specs["data_type"] = "fp32"
del specs["batch_size"]
del specs["batches"]

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/export"

response = requests.post(endpoint,data=data, headers=headers)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

In [ ]:
# Run action
parent = job_map["train"]
actions = ["export"]
data = json.dumps({"job":parent,"actions":actions})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

job_map["export_fp32"] = response.json()[0]
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
job_id = job_map['export_fp32']
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    print(response)
    print(response.json())
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

### Export: INT8 <a class="anchor" id="head-16"></a>

- Note - if another export job is in Pending state for this model experiment, modifying specs would apply to the Pending experiment

In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/export/schema"

response = requests.get(endpoint, headers=headers)

print(response)
#print(response.json()) ## Uncomment for verbose schema
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Apply changes
specs["data_type"] = "int8"
specs["batches"] = 10
specs["batch_size"] = 4
specs["max_batch_size"] = 4

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/export"

response = requests.post(endpoint,data=data, headers=headers)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

In [ ]:
# Run action
parent = job_map["train"]
actions = ["export"]
data = json.dumps({"job":parent,"actions":actions})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

job_map["export_int8"] = response.json()[0]
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
job_id = job_map['export_int8']
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    print(response)
    print(response.json())
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

In [ ]:
# Download job contents once the above job shows "Done" status
job_id = job_map["export_int8"]
endpoint = f'{base_url}/model/{model_id}/job/{job_id}/download'

# Save
temptar = f'{job_id}.tar.gz'
with requests.get(endpoint, headers=headers, stream=True) as r:
    r.raise_for_status()
    with open(temptar, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

print("Untarring")
# Untar to destination
tar_command = f'tar -xvf {temptar} -C {workdir}/'
os.system(tar_command)
os.remove(temptar)
print(f"Results at {workdir}/{job_id}")
model_downloaded_path = f"{workdir}/{job_id}"

In [ ]:
# Look for the generated .etlt file, tensorrt .engine file and cal.bin calibration cache file
!ls {model_downloaded_path}

### Model convert using TAO-Converter <a class="anchor" id="head-17"></a>

- Here, we use the INT8 exported model to convert to target platform

In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/convert/schema"

response = requests.get(endpoint, headers=headers)

print(response)
#print(response.json()) ## Uncomment for verbose schema
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Apply changes
specs["t"] = "int8"
specs["b"] = 8
specs["m"] = 16
specs["d"] = "3,80,60"
specs["i"] = "nchw"
specs["o"] = "base_color/Softmax,category/Softmax,season/Softmax"

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/convert"

response = requests.post(endpoint,data=data, headers=headers)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

In [ ]:
# Run action
parent = job_map["export_int8"]
actions = ["convert"]
data = json.dumps({"job":parent,"actions":actions})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

job_map["model_convert"] = response.json()[0]
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
job_id = job_map['model_convert']
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    print(response)
    print(response.json())
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

### TAO inference <a class="anchor" id="head-18"></a>

- Run inference on a set of images using the .tlt model created at train step

In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/inference/schema"

response = requests.get(endpoint, headers=headers)

print(response)
#print(response.json()) ## Uncomment for verbose schema
specs = response.json()["default"]
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Apply changes to the specs you want to modify

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/inference"

response = requests.post(endpoint,data=data, headers=headers)

print(response)
print(json.dumps(response.json(), sort_keys=True, indent=4))

In [ ]:
# Run action
parent = job_map["train"]
actions = ["inference"]
data = json.dumps({"job":parent,"actions":actions})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

job_map["inference_tlt"] = response.json()[0]
print(job_map)

In [ ]:
# Monitor job status by repeatedly running this cell
job_id = job_map['inference_tlt']
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    print(response)
    print(response.json())
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

In [ ]:
# Download job contents once the above job shows "Done" status
job_id = job_map["inference_tlt"]
endpoint = f'{base_url}/model/{model_id}/job/{job_id}/download'

# Save
temptar = f'{job_id}.tar.gz'
with requests.get(endpoint, headers=headers, stream=True) as r:
    r.raise_for_status()
    with open(temptar, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

print("Untarring")
# Untar to destination
tar_command = f'tar -xvf {temptar} -C {workdir}/'
os.system(tar_command)
os.remove(temptar)
print(f"Results at {workdir}/{job_id}")
inference_out_path = f"{workdir}/{job_id}"

In [ ]:
# Inference output must be here
!ls {inference_out_path}/

In [ ]:
# Print Classification results
!cat {inference_out_path}/result.txt